In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 as sq

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input/import"]).decode("utf8"))

In [ ]:
con = sq.connect("../input/import/database.sqlite")
match = pd.read_sql_query("SELECT * from Match", con)
match = match[['id','match_api_id','home_team_api_id', 'home_team_goal', 'away_team_goal','away_team_api_id','B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD',
              'LBA']]

In [ ]:
match

In [ ]:
match.dropna()

In [ ]:
match.loc[(match['home_team_goal'] > match['away_team_goal']),'home_win']= int('1')
match.loc[(match['home_team_goal'] < match['away_team_goal']),'away_win']= int('1')
match.loc[(match['home_team_goal'] == match['away_team_goal']),'draw']= int('1')
match

In [ ]:
# replace NaN with 0
match.fillna(0, inplace=True)
match

In [ ]:
match = match[match.B365H != 0]
match = match[match.BWH != 0]
match = match[match.IWD != 0]
match = match[match.LBH != 0]

In [ ]:
match2=match
# if we dont convert odds to probs the accuracy goes approx 50% up
match2['B365H'] = 1/match['B365H']
match2['B365D'] = 1/match['B365D']
match2['B365A'] = 1/match['B365A']
match2['BWH'] = 1/match['BWH']
match2['BWD'] = 1/match['BWD']
match2['BWA'] = 1/match['BWA']
match2['IWH'] = 1/match['IWH']
match2['IWD'] = 1/match['IWD']
match2['IWA'] = 1/match['IWA']
match2['LBH'] = 1/match['LBH']
match2['LBD'] = 1/match['LBD']
match2['LBA'] = 1/match['LBA']
match2

In [ ]:
match.loc[(match['home_win'] == 1),'result']= 'home_win'
match.loc[(match['away_win'] == 1),'result']= 'away_win'
match.loc[(match['draw'] == 1),'result']= 'draw'
match.loc[(match['home_team_goal'] > match['away_team_goal']),'home_win'] = int('1')
match.loc[(match['home_team_goal'] < match['away_team_goal']),'away_win'] = int('1')
match.loc[(match['home_team_goal'] == match['away_team_goal']),'draw'] = int('1')
match = match2[match2.result != 0]

cmeans_data = match[['B365H', 'B365D', 'B365A','result']]
cmeans_data = cmeans_data[cmeans_data.result != 0]
cmeans_data = cmeans_data[cmeans_data.B365H != 0]
cmeans_data = cmeans_data[cmeans_data.B365D != 0]
cmeans_data = cmeans_data[cmeans_data.B365A != 0]
cmeans_data.to_csv(r'soccer_data_cmeans.csv', index = False)
cmeans_data

In [ ]:
match

In [ ]:
cmeans_data_BW = match[['BWH', 'BWD', 'BWA','result']]
cmeans_data_BW

cmeans_data_BW = cmeans_data_BW[cmeans_data_BW.BWH != 0]
cmeans_data_BW = cmeans_data_BW[cmeans_data_BW.BWD != 0]
cmeans_data_BW = cmeans_data_BW[cmeans_data_BW.BWA != 0]
#cmeans_data_BW

Οι πίνακες χαρακτηρισιτκών για τις υπόλοιπες 3 εταιρείες

In [ ]:
cmeans_data_BW = match[['BWH', 'BWD', 'BWA','result']]
cmeans_data_IW = match[['IWH', 'IWD', 'IWA','result']]
cmeans_data_LB = match[['LBH', 'LBD', 'LBA','result']]
#cmeans_data_BW

In [ ]:
from subprocess import check_output
print(check_output(["ls", "/kaggle/working"]).decode("utf8"))

In [ ]:
import random
import operator
import math

df_full = pd.read_csv('/kaggle/working/soccer_data_cmeans.csv') 
#df_full.sort_values(by=['result'], inplace=True)
#df_full # exei kai stili result

In [ ]:
columns = list(df_full.columns)
features = columns[:len(columns)-1] # company prediction for 'away_win', 'draw' and 'home_win'
class_labels = list(df_full[columns[-1]]) # label of every match ('away_win', 'draw', 'home_win')
df = df_full[features]
# df #mono ta odds χωρις results
# class_labels

---------------------


https://pypi.org/project/fuzzy-c-means/ na ftiaxw df1 df2 df3 gia ka8e etaireia na brw ta kentra

In [ ]:
! pip install fuzzy-c-means

In [ ]:
df_DF=df # κρατα ενα copy των προγνωστικών
df=df.to_numpy() # ο αλγοριθμος cmeans πέρνει εισοδο ενα array

# κανε drop την τελευταια στηλη απο κάθε πίνακα εταιρίας

In [ ]:
df_BW= cmeans_data_BW
df_IW= cmeans_data_IW 
df_LB= cmeans_data_LB

columns = list(df_BW.columns)
featuresBW = columns[:len(columns)-1] # company prediction for 'away_win', 'draw' and 'home_win'
class_labelsBW = list(df_BW[columns[-1]]) # label of every match ('away_win', 'draw', 'home_win')
columns = list(df_IW.columns)
featuresIW = columns[:len(columns)-1] # company prediction for 'away_win', 'draw' and 'home_win'
class_labelsIW = list(df_IW[columns[-1]]) # label of every match ('away_win', 'draw', 'home_win')
columns = list(df_LB.columns)
featuresLB = columns[:len(columns)-1] # company prediction for 'away_win', 'draw' and 'home_win'
class_labelsLB = list(df_LB[columns[-1]]) # label of every match ('away_win', 'draw', 'home_win')


df_BW = df_BW[featuresBW]
df_IW = df_IW[featuresIW]
df_LB = df_LB[featuresLB]
print (df_BW) 
print (df_IW)
print (df_LB)

In [ ]:
#df

In [ ]:
# cmeans_data_BW

In [ ]:
#df_full # περιέχει στηλη result


# clustering για την B365

In [ ]:

from fcmeans import FCM
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt
from seaborn import scatterplot as scatter
from matplotlib import cm
from mpl_toolkits.mplot3d.axes3d import get_test_data


# create artifitial dataset με κέντρα τα centers
n_samples = 5000
n_bins = 3  # use 3 bins for calibration_curve as we have 3 clusters here
centers = [(-5, -5, -5), (0, 0, 0), (5, 5, 5)]
# centers = [(a,b), (g,d), (e,z)], where: a>b -> Win, g=d -> Draw, e<z -> Defeat
# αυτο φτιανχψει τυχαίες κατανομες (πινακα 3 στηλων) αρα βάζουμε ολες τις εταιρίες στον ίδιο πίνακα?
#X,_ = make_blobs(n_samples=n_samples, n_features=3, cluster_std=1.0,
 #                 centers=centers, shuffle=False, random_state=42)
X= df
    # fit the fuzzy-c-means
fcm = FCM(n_clusters=3)
fcm.fit(X)

# outputs // αυτο φτιαχνει τα τελικά clusters
fcm_centers = fcm.centers
fcm_labels  = fcm.u.argmax(axis=1)
fcm_centers

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d') # afto einai gia 3d 
xs =X[:,0].flatten()
ys = X[:,1].flatten()
zs = X[:,2].flatten()
#fig = plt.figure()

fig = plt.figure(figsize=(10,10))

ax= fig.add_subplot(111, projection='3d') # afto einai gia 3d 
#ax= plt.scatter(figsize=(8,5))
ax.scatter(xs,ys,zs, s=10, c=fcm_labels)
#ax.lmplot ( x="sepal_length", y="sepal_width", data=df, fit_reg=False, hue='species', legend=False, markers=["o", "x", "1"])
ax.scatter(fcm_centers[:,0], fcm_centers[:,1],fcm_centers[:,2],marker="s",s=200)  # αυτο πρεπει να μπει δινει τα κεντρα

ax.view_init(30, 185)
ax.set_xlabel("Home win")
ax.set_ylabel("Draw")
ax.set_zlabel("Away win")
ax.set_title("Soccer odds clustering - B365")

plt.show()

## edw ta ypoloipa apo to lib https://github.com/omadson/fuzzy-c-means/blob/master/fcmeans/fcm.py





In [ ]:
print("Membership degrees matrix for B365")
print(fcm.u)# h καθε στηλη ειναι το membership degree (cluster1, cluster2, cluster 3) για κάθε set (γραμμη / αγωνα)
print("Hard membership matrix")
print(fcm.predict(X))
print("Cluster centers: ")
print(fcm_centers)
fcm_labels = fcm.u.argmax(axis=1)

In [ ]:

# εδω να προστεθει η στήλη fcm_labels στο df_full
df_full["fcm_labels"] = pd.Series(fcm_labels)
#df_full

In [ ]:
cluster0 = df_full[df_full["fcm_labels"]==0]
cluster1 = df_full[df_full["fcm_labels"]==1]
cluster2 = df_full[df_full["fcm_labels"]==2]

curr_cluster = 0
clusters = [cluster0, cluster1, cluster2]
for cluster in clusters:
    away_win_df = cluster[cluster["result"]=="away_win"]
    home_win_df = cluster[cluster["result"]=="home_win"]
    draw_df = cluster[cluster["result"]=="draw"]
    
    total_away_wins = away_win_df.shape[0]
    total_home_wins = home_win_df.shape[0]
    total_draws = draw_df.shape[0]

    total = total_away_wins + total_home_wins + total_draws
    f0 = (total_away_wins / total) * 100
    f1 = (total_home_wins / total) * 100
    f2 = (total_draws / total) * 100
    print("cluster-{} has {} home_wins {} away_wins and {} draws".format(curr_cluster, total_home_wins, total_away_wins, total_draws))
    print("home_wins% = {}, away_wins% = {}, draws% = {}".format(f0, f1, f2))
    
    curr_cluster += 1

# clustering για την IW

In [ ]:
# df_IW

In [ ]:
X= df_IW.to_numpy()
    # fit the fuzzy-c-means
fcm = FCM(n_clusters=3)
fcm.fit(X)

# outputs // αυτο φτιαχνει τα τελικά clusters
fcm_centers = fcm.centers
fcm_labels  = fcm.u.argmax(axis=1)
fcm_centers

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d') # afto einai gia 3d 
xs =X[:,0].flatten()
ys = X[:,1].flatten()
zs = X[:,2].flatten()
#fig = plt.figure()

fig = plt.figure(figsize=(10,10))

ax= fig.add_subplot(111, projection='3d') # afto einai gia 3d 
#ax= plt.scatter(figsize=(8,5))
ax.scatter(xs,ys,zs, s=10, c=fcm_labels)
#ax.lmplot ( x="sepal_length", y="sepal_width", data=df, fit_reg=False, hue='species', legend=False, markers=["o", "x", "1"])
ax.scatter(fcm_centers[:,0], fcm_centers[:,1],fcm_centers[:,2],marker="s",s=200)  # αυτο πρεπει να μπει δινει τα κεντρα

ax.view_init(30, 185)
ax.set_xlabel("Home win")
ax.set_ylabel("Draw")
ax.set_zlabel("Away win")
ax.set_title("Soccer odds clustering - IW")

plt.show()

In [ ]:
print("Membership degrees matrix for IW")
print(fcm.u)# h καθε στηλη ειναι το membership degree (cluster1, cluster2, cluster 3) για κάθε set (γραμμη / αγωνα)
print("Hard membership matrix")
print(fcm.predict(X))
print("Cluster centers: ")
print(fcm_centers)

In [ ]:
# fcm_labels.size

In [ ]:
# cmeans_data_IW[22465:] 

In [ ]:
# df_full

In [ ]:
 #cmeans_data_IW

In [ ]:
cmeans_data_IW["fcm_labels"] = pd.Series(fcm_labels)
#cmeans_data_IW["fcm_labels"].dropna
#cmeans_data_IW["fcm_labels"].astype(int)

In [ ]:
cluster0 = cmeans_data_IW[cmeans_data_IW["fcm_labels"]==0]
cluster1 = cmeans_data_IW[cmeans_data_IW["fcm_labels"]==1]
cluster2 = cmeans_data_IW[cmeans_data_IW["fcm_labels"]==2]

curr_cluster = 0
clusters = [cluster0, cluster1, cluster2]
for cluster in clusters:
    away_win_df = cluster[cluster["result"]=="away_win"]
    home_win_df = cluster[cluster["result"]=="home_win"]
    draw_df = cluster[cluster["result"]=="draw"]
    
    total_away_wins = away_win_df.shape[0]
    total_home_wins = home_win_df.shape[0]
    total_draws = draw_df.shape[0]

    total = total_away_wins + total_home_wins + total_draws
    f0 = (total_away_wins / total) * 100
    f1 = (total_home_wins / total) * 100
    f2 = (total_draws / total) * 100
    print("cluster-{} has {} home_wins {} away_wins and {} draws".format(curr_cluster, total_home_wins, total_away_wins, total_draws))
    print("home_wins% = {}, away_wins% = {}, draws% = {}".format(f0, f1, f2))
    
    curr_cluster += 1

# clustering για την BW

In [ ]:
X= df_BW.to_numpy()
    # fit the fuzzy-c-means
fcm = FCM(n_clusters=3)
fcm.fit(X)

# outputs // αυτο φτιαχνει τα τελικά clusters
fcm_centers = fcm.centers
fcm_labels  = fcm.u.argmax(axis=1)
fcm_centers

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d') # afto einai gia 3d 
xs =X[:,0].flatten()
ys = X[:,1].flatten()
zs = X[:,2].flatten()
#fig = plt.figure()

fig = plt.figure(figsize=(10,10))

ax= fig.add_subplot(111, projection='3d') # afto einai gia 3d 
#ax= plt.scatter(figsize=(8,5))
ax.scatter(xs,ys,zs, s=10, c=fcm_labels)
#ax.lmplot ( x="sepal_length", y="sepal_width", data=df, fit_reg=False, hue='species', legend=False, markers=["o", "x", "1"])
ax.scatter(fcm_centers[:,0], fcm_centers[:,1],fcm_centers[:,2],marker="s",s=200)  # αυτο πρεπει να μπει δινει τα κεντρα

ax.view_init(30, 185)
ax.set_xlabel("Home win")
ax.set_ylabel("Draw")
ax.set_zlabel("Away win")
ax.set_title("Soccer odds clustering - BW")

plt.show()

In [ ]:
print("Membership degrees matrix for BW")
print(fcm.u)# h καθε στηλη ειναι το membership degree (cluster1, cluster2, cluster 3) για κάθε set (γραμμη / αγωνα)
print("Hard membership matrix")
print(fcm.predict(X))
print("Cluster centers: ")
print(fcm_centers)

In [ ]:
cmeans_data_BW["fcm_labels"] = pd.Series(fcm_labels)

In [ ]:
cluster0 = cmeans_data_BW[cmeans_data_IW["fcm_labels"]==0]
cluster1 = cmeans_data_BW[cmeans_data_IW["fcm_labels"]==1]
cluster2 = cmeans_data_BW[cmeans_data_IW["fcm_labels"]==2]

curr_cluster = 0
clusters = [cluster0, cluster1, cluster2]
for cluster in clusters:
    away_win_df = cluster[cluster["result"]=="away_win"]
    home_win_df = cluster[cluster["result"]=="home_win"]
    draw_df = cluster[cluster["result"]=="draw"]
    
    total_away_wins = away_win_df.shape[0]
    total_home_wins = home_win_df.shape[0]
    total_draws = draw_df.shape[0]

    total = total_away_wins + total_home_wins + total_draws
    f0 = (total_away_wins / total) * 100
    f1 = (total_home_wins / total) * 100
    f2 = (total_draws / total) * 100
    print("cluster-{} has {} home_wins {} away_wins and {} draws".format(curr_cluster, total_home_wins, total_away_wins, total_draws))
    print("home_wins% = {}, away_wins% = {}, draws% = {}".format(f0, f1, f2))
    
    curr_cluster += 1

# clustering για την LB

In [ ]:
df_LB.dropna

In [ ]:
X= df_LB.to_numpy()
    # fit the fuzzy-c-means
fcm = FCM(n_clusters=3)
fcm.fit(X)

# outputs // αυτο φτιαχνει τα τελικά clusters
fcm_centers_LB = fcm.centers
fcm_labels_LB  = fcm.u.argmax(axis=1)
fcm_centers_LB

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d') # afto einai gia 3d 
xs =X[:,0].flatten()
ys = X[:,1].flatten()
zs = X[:,2].flatten()
#fig = plt.figure()

fig = plt.figure(figsize=(10,10))

ax= fig.add_subplot(111, projection='3d') # afto einai gia 3d 
#ax= plt.scatter(figsize=(8,5))
ax.scatter(xs,ys,zs, s=10, c=fcm_labels)
#ax.lmplot ( x="sepal_length", y="sepal_width", data=df, fit_reg=False, hue='species', legend=False, markers=["o", "x", "1"])
ax.scatter(fcm_centers[:,0], fcm_centers[:,1],fcm_centers[:,2],marker="s",s=200)  # αυτο πρεπει να μπει δινει τα κεντρα

ax.view_init(30, 185)
ax.set_xlabel("Home win")
ax.set_ylabel("Draw")
ax.set_zlabel("Away win")
ax.set_title("Soccer odds clustering - LB")

plt.show()

In [ ]:
print("Membership degrees matrix for LB")
print(fcm.u)# h καθε στηλη ειναι το membership degree (cluster1, cluster2, cluster 3) για κάθε set (γραμμη / αγωνα)
print("Hard membership matrix")
print(fcm.predict(X))
print("Cluster centers: ")
print(fcm_centers_LB)

In [ ]:
fcm.u # h καθε στηλη ειναι το membership degree (cluster1, cluster2, cluster 3) για κάθε set (γραμμη / αγωνα)


In [ ]:
print("Cluster centers: ",fcm_centers_LB)

In [ ]:
print("Hard Memership values - cluster no: ",fcm_labels_LB)
# prepei na brw ta apotelesmata που αντιστοιχουν σε κάθε αριθμό cluster και να μετρήσω το συνολο του καθε αποτελεσματος σε κάθε cluster

In [ ]:
df_LB["fcm_labels"]=pd.Series(fcm_labels)
df_LB

In [ ]:
cmeans_data_LB["fcm_labels"] = pd.Series(fcm_labels)

In [ ]:
cluster0 = cmeans_data_LB[cmeans_data_LB["fcm_labels"]==0]
cluster1 = cmeans_data_LB[cmeans_data_LB["fcm_labels"]==1]
cluster2 = cmeans_data_LB[cmeans_data_LB["fcm_labels"]==2]

In [ ]:
curr_cluster = 0
clusters = [cluster0, cluster1, cluster2]
for cluster in clusters:
    away_win_df = cluster[cluster["result"]=="away_win"]
    home_win_df = cluster[cluster["result"]=="home_win"]
    draw_df = cluster[cluster["result"]=="draw"]
    
    total_away_wins = away_win_df.shape[0]
    total_home_wins = home_win_df.shape[0]
    total_draws = draw_df.shape[0]

    total = total_away_wins + total_home_wins + total_draws
    f0 = (total_away_wins / total) * 100
    f1 = (total_home_wins / total) * 100
    f2 = (total_draws / total) * 100
    print("cluster-{} has {} home_wins {} away_wins and {} draws".format(curr_cluster, total_home_wins, total_away_wins, total_draws))
    print("home_wins% = {}, away_wins% = {}, draws% = {}".format(f0, f1, f2))
    
    curr_cluster += 1

In [ ]:
#df_full['cluster'] = fcm_labels  # πρεπει να προσθεσω στηλη με τον αριθμό του cluster διπλα σε καθε αποτέλεμα
#df_full.shape


In [ ]:
win_occurrences = np.sum(fcm_labels==0)
draw_occurrences = np.sum(fcm_labels==2)
away_occurrences = np.sum(fcm_labels==1)

In [ ]:
print ("cluster1 περιέχει:", win_occurrences)
print ("cluster2 περιέχει:", away_occurrences)
print ("cluster3 περιέχει:", draw_occurrences)